In [1]:
import re
import pymongo
from pymongo import MongoClient
import numpy as np
from matplotlib import pyplot as plt
from firebase import firebase
import json
import pickle
import math
import usaddress
import xml.etree.ElementTree as ET
from lxml import etree
import geopy
from geopy import distance
import time
from shapely.geometry import Polygon as pg
from shapely.geometry import Point
from decimal import *

In [5]:
file_in="/home/allan/Desktop/new_haven_seeclickfix_visualization/data/new_haven.xml"
data = etree.parse(open(file_in))

In [2]:
with open("street_coords.pkl","r") as f:
    street_coords = pickle.load(f)

In [3]:
with open("/home/allan/Desktop/new_haven_seeclickfix_visualization/data/new_haven.geojson","r") as f:
    neighborhood_data=json.load(f)

In [6]:
## ways were obtained by searching "new haven" on the OSM website
nh_ways=["41247749","42446923","42446898","41759002","41760815","41759037","41760819","350714192","41744416","41439114"]
nh_bounds=[]
for way in nh_ways:
    nodes=data.find("./way[@id='"+way+"']")
    coords=[]
    for nd in nodes.iter("nd"):
        ref=nd.attrib["ref"]
        node=data.find("./node[@id='"+ref+"']")
        coord=[float(node.attrib["lon"]),float(node.attrib["lat"])]
        coords.append(coord)
    if len(nh_bounds)>0:
        last_coord=nh_bounds[len(nh_bounds)-1]
        if coord[0]==last_coord[0] and coord[1]==last_coord[1]:
            coords.reverse()
    nh_bounds+=coords

In [14]:
with open("nh_bounds.pkl","w") as f:
    pickle.dump(nh_bounds,f)

In [7]:
poly=pg(*[nh_bounds])
#plt.figure()
street_nhv={}
for name in street_coords.keys():
    street_nhv[name]=[]
    for i in range(len(street_coords[name])):
        street_nhv[name].append([])
        for j in range(len(street_coords[name][i])):
            innh=False
            if poly.contains(Point((float(street_coords[name][i][j][0]),float(street_coords[name][i][j][1])))):
                innh=True
                #plt.plot(float(street_coords[name][i][j][0]),float(street_coords[name][i][j][1]),'bo')
            #else:
                #plt.plot(float(street_coords[name][i][j][0]),float(street_coords[name][i][j][1]),'ro')
            street_nhv[name][i].append(innh)
#for coord in nh_bounds:
    #plt.plot(coord[0],coord[1],'ko')
#plt.show()

In [9]:
## here we are checking to see if 
## any street segments switch back 
## and forth between new haven and 
## not new haven

twice=0
nsegments=0
for name,segments in street_nhv.items():
    q=0
    for segment in segments:
        once=False
        t=False
        nsegments+=1
        for i in range(1,len(segment)):
            if not segment[i]==segment[i-1]:
                if once:
                    if not t:
                        twice+=1
                        t=True
                once=True
        if once and len(segment)<3:
            print segment,name,q
        if t:
            print segment,name,q
        q+=1
street_nhv["Dell Drive"][0][6]=True #change this so that we can do length calculation more easily...

[True, False] Spring Street 0
[True, False] South End Road 2
[False, True] Cherry Ann Street 0
[True, False] Orange Avenue 1


In [12]:
street_lengths={}
for name,segments in street_coords.items():
    street_length=0
    for i in range(len(segments)):
        coords = np.array(segments[i])[np.array(street_nhv[name][i]).nonzero()]
        if len(coords)>1:
            for j in range(1,len(coords)):
                street_length += geopy.distance.vincenty(tuple(coords[j]), tuple(coords[j-1])).miles
    if street_length>0.:
        street_lengths[name]=street_length

In [13]:
with open("street_lengths.pkl","w") as f:
    pickle.dump(street_lengths,f)

In [4]:
client=MongoClient()
db=client.nh4
with open("street_lengths.pkl","r") as f:
    street_lengths=pickle.load(f)
for name,length in street_lengths.items():
    db.street_lengths.insert_one({name.lower():length})